In [ ]:
import sys

sys.path[1:1] = ["_common", "_common/qiskit"]
sys.path[1:1] = ["../../_common", "../../_common/qiskit"]
sys.path[1:1] = ["../qiskit"]
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import time
import math
import numpy as np
np.random.seed(0)
import execute as ex
import metrics as metrics
from collections import defaultdict

In [ ]:
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import TimeEvolutionProblem, SciPyRealEvolver

# Import the hamiltonian simualtion circuit from our qiskit benchmark. 
from hamiltonian_simulation_benchmark import HamiltonianSimulation

In [ ]:
# The functions in this cell are used to exactly simulate the hamiltonian evolution using a classical matrix evolution. 

def initial_state(n_spins: int, initial_state: str = "checker") -> QuantumCircuit:
    """
    Initialize the quantum state.

    Dev note: This function is copy/pasted from HamiltonianSimulation.
    
    Args:
        n_spins (int): Number of spins (qubits).
        initial_state (str): The chosen initial state. By default applies the checkerboard state, but can also be set to "ghz", the GHZ state.

    Returns:
        QuantumCircuit: The initialized quantum circuit.
    """
    qc = QuantumCircuit(n_spins)

    if initial_state.strip().lower() == "checkerboard" or initial_state.strip().lower() == "neele":
        # Checkerboard state, or "Neele" state
        for k in range(0, n_spins, 2):
            qc.x([k])
    elif initial_state.strip().lower() == "ghz":
        # GHZ state: 1/sqrt(2) (|00...> + |11...>)
        qc.h(0)
        for k in range(1, n_spins):
            qc.cx(k-1, k)

    return qc

def construct_TFIM_hamiltonian(n_spins: int) -> SparsePauliOp:
    """
    Construct the Transverse Field Ising Model (TFIM) Hamiltonian.

    Args:
        n_spins (int): Number of spins (qubits).

    Returns:
        SparsePauliOp: The Hamiltonian represented as a sparse Pauli operator.
    """
    pauli_strings = []
    coefficients = []
    g = 0.2  # Strength of the transverse field

    # Pauli spin vector product terms
    for i in range(n_spins):
        x_term = 'I' * i + 'X' + 'I' * (n_spins - i - 1)
        pauli_strings.append(x_term)
        coefficients.append(g)

    identity_string = ['I'] * n_spins

    # ZZ operation on each pair of qubits in a linear chain
    for j in range(2):
        for i in range(j % 2, n_spins - 1, 2):
            zz_term = identity_string.copy()
            zz_term[i] = 'Z'
            zz_term[(i + 1) % n_spins] = 'Z'
            zz_term = ''.join(zz_term)
            pauli_strings.append(zz_term)
            coefficients.append(1.0)

    return SparsePauliOp.from_list(zip(pauli_strings, coefficients))

def construct_heisenberg_hamiltonian(n_spins: int, w: int, hx: list[float], hz: list[float]) -> SparsePauliOp:
    """
    Construct the Heisenberg Hamiltonian with disorder.

    Args:
        n_spins (int): Number of spins (qubits).
        w (float): Strength of two-qubit interactions for heisenberg hamiltonian. 
        hx (list[float]): Strength of internal disorder parameter for heisenberg hamiltonian. 
        hz (list[float]): Strength of internal disorder parameter for heisenberg hamiltonian. 

    Returns:
        SparsePauliOp: The Hamiltonian represented as a sparse Pauli operator.
    """

    pauli_strings = []
    coefficients = []

    # Disorder terms
    for i in range(n_spins):
        x_term = 'I' * i + 'X' + 'I' * (n_spins - i - 1)
        z_term = 'I' * i + 'Z' + 'I' * (n_spins - i - 1)
        pauli_strings.append(x_term)
        coefficients.append(w * hx[i])
        pauli_strings.append(z_term)
        coefficients.append(w * hz[i])

    identity_string = ['I'] * n_spins

    # Interaction terms
    for j in range(2):
        for i in range(j % 2, n_spins - 1, 2):
            xx_term = identity_string.copy()
            yy_term = identity_string.copy()
            zz_term = identity_string.copy()

            xx_term[i] = 'X'
            xx_term[(i + 1) % n_spins] = 'X'

            yy_term[i] = 'Y'
            yy_term[(i + 1) % n_spins] = 'Y'

            zz_term[i] = 'Z'
            zz_term[(i + 1) % n_spins] = 'Z'

            pauli_strings.append(''.join(xx_term))
            coefficients.append(1.0)
            pauli_strings.append(''.join(yy_term))
            coefficients.append(1.0)
            pauli_strings.append(''.join(zz_term))
            coefficients.append(1.0)

    return SparsePauliOp.from_list(zip(pauli_strings, coefficients))

def construct_hamiltonian(n_spins: int, hamiltonian: str, w: float, hx : list[float], hz: list[float]) -> SparsePauliOp:
    """
    Construct the Hamiltonian based on the specified method.

    Args:
        n_spins (int): Number of spins (qubits).
        hamiltonian (str): Which hamiltonian to run. "Heisenberg" by default but can also choose "TFIM". 
        w (float): Strength of two-qubit interactions for heisenberg hamiltonian. 
        hx (list[float]): Strength of internal disorder parameter for heisenberg hamiltonian. 
        hz (list[float]): Strength of internal disorder parameter for heisenberg hamiltonian. 

    Returns:
        SparsePauliOp: The constructed Hamiltonian.
    """

    hamiltonian = hamiltonian.strip().lower()

    if hamiltonian == "heisenberg":
        return construct_heisenberg_hamiltonian(n_spins, w, hx, hz)
    elif hamiltonian == "tfim":
        return construct_TFIM_hamiltonian(n_spins)
    else:
        raise ValueError("Invalid Hamiltonian specification.")

def HamiltonianSimulationExact(n_spins: int, t: float, init_state: str, hamiltonian: str, w: float, hx: list[float], hz: list[float]) -> dict:
    """
    Perform exact Hamiltonian simulation using classical matrix evolution.

    Args:
        n_spins (int): Number of spins (qubits).
        t (float): Duration of simulation.
        init_state (str): The chosen initial state. By default applies the checkerboard state, but can also be set to "ghz", the GHZ state.
        hamiltonian (str): Which hamiltonian to run. "Heisenberg" by default but can also choose "TFIM". 
        w (float): Strength of two-qubit interactions for heisenberg hamiltonian. 
        hx (list[float]): Strength of internal disorder parameter for heisenberg hamiltonian. 
        hz (list[float]): Strength of internal disorder parameter for heisenberg hamiltonian. 

    Returns:
        dict: The distribution of the evolved state.
    """
    hamiltonian_sparse = construct_hamiltonian(n_spins, hamiltonian, w, hx, hz)
    time_problem = TimeEvolutionProblem(hamiltonian_sparse, t, initial_state=initial_state(n_spins, init_state))
    result = SciPyRealEvolver(num_timesteps=1).evolve(time_problem)
    return result.evolved_state.probabilities_dict()

In [ ]:
import json
from qiskit_aer import Aer
from qiskit import transpile

w = 1 
k = 5
t = .2 # Heisenberg state evolves "just" enough 

min_qubits = 2
max_qubits = 12

backend = Aer.get_backend("qasm_simulator")

precalculated_data = {}

# store parameters in precalculated data
precalculated_data["w"] = w
precalculated_data["k"] = k
precalculated_data["t"] = t

# add parameter random values to precalculated data to ensure consistency
np.random.seed(26)
precalculated_data['hx'] = list(2 * np.random.random(20) - 1) # random numbers between [-1, 1]
np.random.seed(75)
precalculated_data['hz'] = list(2 * np.random.random(20) - 1) # random numbers between [-1, 1]

num_shots = 100000

for n_spins in range(min_qubits, max_qubits+1):

        print(f"Now running n_spins {n_spins}")

        hx = precalculated_data['hx'][:n_spins]
        hz = precalculated_data['hz'][:n_spins]

        qc = HamiltonianSimulation(n_spins, k, t, hamiltonian="heisenberg", w=w, hx = hx, hz = hz)

        qc3 = HamiltonianSimulation(n_spins, k, t, hamiltonian="tfim", w=w, hx = hx, hz = hz)

        transpiled_qc = transpile(qc, backend, optimization_level=0)
        job = backend.run(transpiled_qc, shots=num_shots)
        result = job.result()
        counts = result.get_counts(qc)

        transpiled_qc3 = transpile(qc3, backend, optimization_level=0)
        job3 = backend.run(transpiled_qc3, shots=num_shots)
        result3 = job3.result()
        counts3 = result3.get_counts()

        dist = {}
        for key in counts.keys():
            prob = counts[key] / num_shots
            dist[key] = prob

        dist3 = {}
        for key in counts3.keys():
            prob = counts3[key] / num_shots
            dist3[key] = prob

        # add dist values to precalculated data for use in fidelity calculation
        precalculated_data[f"Heisenberg - Qubits{n_spins}"] = dist  
        precalculated_data[f"Exact Heisenberg - Qubits{n_spins}"] = HamiltonianSimulationExact(n_spins, t=t, init_state = "checkerboard", hamiltonian="heisenberg", w=w, hx = hx, hz = hz)
        precalculated_data[f"TFIM - Qubits{n_spins}"] = dist3 
        precalculated_data[f"Exact TFIM - Qubits{n_spins}"] = HamiltonianSimulationExact(n_spins, t=t, init_state = "ghz", hamiltonian="tfim", w=w, hx = hx, hz = hz) 

# https://stackoverflow.com/questions/36021332/how-to-prettyprint-human-readably-print-a-python-dict-in-json-format-double-q
with open('precalculated_data.json', 'w') as f:
    f.write(json.dumps(
        precalculated_data,
        sort_keys=True,
        indent=4,
        separators=(',', ': ')
        ))